# Assignment 2: Data Preparation
## Nathan Tyler
### 2/18/26

Purpose: This script is designed to serve the retail client by taking their raw product listing data, consolidating the data from multiple sources into a standardized, clean, and validated data set to be used for futher analysis by downstream users. 

## Part A: Data audit (profiling)

In [13]:
#Load the retail_dirty_210.csv into a Pandas DataFrame. @ https://raw.githubusercontent.com/ttchuang/dataset/refs/heads/master/
import pandas as pd
    
df = pd.read_csv('https://raw.githubusercontent.com/ttchuang/dataset/refs/heads/master/retail_dirty_210.csv')
#Provide information about DF, including data types and non-null counts, and show the first few rows.
df.info()
df.head()
df.tail()
print(df.head())
print("---------------------------------------------------------------")
print(df.tail(10))
#Count unique values
Sumstatsdf = df.describe()
print(Sumstatsdf)
#Find the percentage of missing values in each column.
missing_percentage = df.isnull().mean() * 100
print(missing_percentage)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product     205 non-null    object
 1   size        203 non-null    object
 2   price       184 non-null    object
 3   in_stock    194 non-null    object
 4   category    210 non-null    object
 5   date_added  197 non-null    object
dtypes: object(6)
memory usage: 10.0+ KB
            product   size   price in_stock     category   date_added
0         red shirt     XL  $47.59  unknown         Tops   2026-04-06
1   leather wallet       M   35.45    FALSE  Accessories   2025-05-22
2        blue pants     XL    56.9    FALSE      Bottoms  19-Sep-2024
3      sports socks      M   17.85      Yes  Accessories   05/30/2025
4      white tuxedo  Large     NaN        1       Formal  02-Dec-2025
---------------------------------------------------------------
             product   size      price in_stock    

# Distinct Data Problems in this set
1. All columns except the "category" colunm have more than 1 missing or null values 
    - product has 5 missing or null values (2.38%)
    - size has 7 missing or null values (3.33%)
    - price has 26 missing or null values (12.38%)
    - in_stock has 16 missing or null values (7.62%)
    - date_added has 13 missing or null values (6.19%)
2. All Data types are objects, when numeric and date time formats are more appropriate 
3. price colunm contains non numeric characters
    - It shows "free" in price instead of being "0"
4. in_stock does not use boolean format, it uses string and has inconsistancies in data entries (mixed cases)
5. date_added is stored as string and not date time 
6. Inconsistancies in sizing
    - It shows there are 24 Unique sizes, which is very unusual, typically there is sizes ranging from XS-XXL (Roughly six would be more appropriate)
7. High cardnality in Price column 
8. The data has white space and capitalization inconsitancies in formatting

# Part B: Cleaning & standardization

In [14]:
#Standardize the sting data by stripping whitespace and converting to a consistent case

df['product'] = df['product'].str.strip().str.lower()
df['size'] = df['size'].str.strip().str.upper()
df['category'] = df['category'].str.strip().str.lower()
df['in_stock'] = df['in_stock'].str.strip().str.lower()
df.head()

#Handle Blanks and N/A
df['product'] = df['product'].replace(['', 'N/A', '??'],  pd.NA)
df['size'] = df['size'].replace(['', 'N/A', '??'], pd.NA)
df['in_stock'] = df['in_stock'].replace(['', 'N/A', '??',], pd.NA)

print(df.head())
print('----------------------------------------------------------------')
print(df.tail())
print('----------------- converted sizing ---------------------------------')
#Convert valid sizes into a consistent set: XS, S, M, L, XL, XXL &  Decide what to do with invalid entries (Large, One Size, N, blanks)
sizing = {
    'XS': 'XS',
    'S': 'S', 
    'M': 'M',
    'L': 'L',
    'XL': 'XL',
    'XXL': 'XXL',
    'LARGE': 'L',
    'EXTRA LARGE': 'XL',
    'MEDIUM': 'M',
    'SMALL': 'S',
    'EXTRA SMALL': 'XS',
    'ONE SIZE': 'OFA',
    'N': pd.NA
}
df['size'] = df['size'].map(sizing)
print(df.head())
print('----------------------------------------------------------------')       
print(df.tail())



          product   size   price in_stock     category   date_added
0       red shirt     XL  $47.59  unknown         tops   2026-04-06
1  leather wallet      M   35.45    false  accessories   2025-05-22
2      blue pants     XL    56.9    false      bottoms  19-Sep-2024
3    sports socks      M   17.85      yes  accessories   05/30/2025
4    white tuxedo  LARGE     NaN        1       formal  02-Dec-2025
----------------------------------------------------------------
             product   size   price in_stock     category  date_added
205       silver tie     XS   60.07    false  accessories  01/01/2026
206       blue pants      L    84.9     true      bottoms  04/02/2025
207   leather wallet      M     NaN    false  accessories  2024-12-27
208      navy blazer    XXL  161.98    false       formal  02/16/2025
209  unknown product  LARGE   26.73       no    outerwear  03/07/2025
----------------- converted sizing ---------------------------------
          product size   price in_stoc

In [15]:
#in_stock standardization
#Convert all values into a single Boolean field:
#TRUE for TRUE, Yes, Y, 1, T, True…
#FALSE for FALSE, No, N, 0, F, false…
#Anything else (unknown, blank) → missing
#in_stock standardization


instock_bool = {
    'TRUE': True,
    'Yes': True,
    'Y': True,
    '1': True,
    'T': True,
    'True': True,
    'FALSE': False,
    'No': False,
    'N': False,
    '0': False,
    'F': False,
    'false': False,
    '': pd.NA,
    'N/A': pd.NA,
    '??': pd.NA,
    'Unknown': pd.NA
}   

df['in_stock'] = df['in_stock'].map(instock_bool)
print(df.head())
print(df.tail())


          product size   price in_stock     category   date_added
0       red shirt   XL  $47.59      NaN         tops   2026-04-06
1  leather wallet    M   35.45    False  accessories   2025-05-22
2      blue pants   XL    56.9    False      bottoms  19-Sep-2024
3    sports socks    M   17.85      NaN  accessories   05/30/2025
4    white tuxedo    L     NaN     True       formal  02-Dec-2025
             product size   price in_stock     category  date_added
205       silver tie   XS   60.07    False  accessories  01/01/2026
206       blue pants    L    84.9      NaN      bottoms  04/02/2025
207   leather wallet    M     NaN    False  accessories  2024-12-27
208      navy blazer  XXL  161.98    False       formal  02/16/2025
209  unknown product    L   26.73      NaN    outerwear  03/07/2025


In [16]:
#Price Cleaning (Most Important)
#Convert to Numeric USD Price
#Strip $, Remove USD, handle comma decimials where possible 

df['price'] = df['price'].apply(lambda x: x.replace('$', '').replace(',', '') if isinstance(x, str) else x).astype(float)

df.head()


ValueError: could not convert string to float: '71.67 USD'